<a href="https://colab.research.google.com/github/Lalith2005reddy/HDR_Image_Segmentation/blob/main/Basic_model_11_09_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from PIL import Image
import os

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Custom Dataset Class
class ExposureDataset(Dataset):
    def __init__(self, img_dir, mask_dir):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.images = os.listdir(img_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx].replace('.jpg', '.png'))

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path)

        # Convert images and masks to tensors
        image = transforms.ToTensor()(image)
        mask = transforms.ToTensor()(mask)

        return image, mask

# Training and Validation Dataset
train_dataset = ExposureDataset('/content/drive/MyDrive/Dataset_phase_4/train_img', '/content/drive/MyDrive/Dataset_phase_4/train_mask')
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Load pre-trained DeepLabv3 model with ResNet-101 backbone
model = models.segmentation.deeplabv3_resnet101(pretrained=True)

# Modify the classifier to match the number of classes (3 in your case: over, under, correctly exposed)
model.classifier[4] = nn.Conv2d(256, 3, kernel_size=(1, 1), stride=(1, 1))
model.to(device)

# Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5) # Using AdamW optimizer

# Learning Rate Scheduler
scheduler = StepLR(optimizer, step_size=10, gamma=0.1) # Reduce learning rate every 10 epochs

# Training Loop
num_epochs = 30
best_loss = float('inf')  # Track the best loss

# Total number of images in the dataset
total_images = len(train_dataset)
print(f"Total number of images: {total_images}")

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for images, masks in train_loader:
        images = images.to(device)
        masks = masks.to(device)

        # Remove the channel dimension from masks
        masks = masks.squeeze(1)

        optimizer.zero_grad()
        outputs = model(images)['out']
        loss = criterion(outputs, masks.long())
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    # Learning rate step
    scheduler.step()

    avg_loss = epoch_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Total images processed: {total_images}")

    # Save the best model
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), 'best_exposure_segmentation_model.pth')
        print("Model saved with lower loss.")

print("Training Complete")

Using device: cuda


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet101_coco-586e9e4e.pth
100%|██████████| 233M/233M [00:01<00:00, 159MB/s]


Total number of images: 311
Epoch [1/30], Loss: 0.8306, Total images processed: 311
Model saved with lower loss.
Epoch [2/30], Loss: 0.4808, Total images processed: 311
Model saved with lower loss.
Epoch [3/30], Loss: 0.3223, Total images processed: 311
Model saved with lower loss.
Epoch [4/30], Loss: 0.2315, Total images processed: 311
Model saved with lower loss.
Epoch [5/30], Loss: 0.1724, Total images processed: 311
Model saved with lower loss.
Epoch [6/30], Loss: 0.1324, Total images processed: 311
Model saved with lower loss.
Epoch [7/30], Loss: 0.1036, Total images processed: 311
Model saved with lower loss.
Epoch [8/30], Loss: 0.0828, Total images processed: 311
Model saved with lower loss.
Epoch [9/30], Loss: 0.0679, Total images processed: 311
Model saved with lower loss.
Epoch [10/30], Loss: 0.0569, Total images processed: 311
Model saved with lower loss.
Epoch [11/30], Loss: 0.0514, Total images processed: 311
Model saved with lower loss.
Epoch [12/30], Loss: 0.0506, Total 